In [158]:
from arcgis.gis import GIS
import sqlalchemy as sqla
import geopandas as gpd
import pandas as pd
import os
import glob
# Get credentials for the jgeo database
import sys
sys.path.append('/home/greg/admin/')
import jgeo_cred as dbcred
# Some stuff we need to access the NMSU ArcGIS Online system
import nmsu_agol_cred # AGOL credentials

# jgeo_utils imports
sys.path.append('/home/greg/GitHub/jgeo_utils')
import jgeo_py.agol as agol
import jgeo_py.pg as pg

# get a connection to jgeo
conn = sqla.create_engine("postgresql+psycopg2://{0}:{1}@{2}:5432/{3}".format(
    dbcred.user, dbcred.pwd, dbcred.host, dbcred.db))

# Log into NMSU AGOL and get a GIS object
gis = GIS(nmsu_agol_cred.agol_url, 
          username=nmsu_agol_cred.agol_user,
          password=nmsu_agol_cred.password)

# Path to local jgeo folder
jgeo_path = '/home/greg/data/rawdata/JornadaGeospatial'

In [40]:
# Get groupids
jgrp_id = gis.groups.search('title:JornadaGIS')[0].id
jgrp_pub_id = gis.groups.search('title:Jornada GIS Portal')[0].id

In [156]:
# Search existing content
my_content = gis.content.search(query="owner:" + gis.users.me.username, 
                                item_type="Feature Layer",
                                max_items=5)
my_content

[<Item title:"Soil_Core_5m" type:Feature Layer Collection owner:gmaurer1>,
 <Item title:"cr_rg_point" type:Feature Layer Collection owner:gmaurer1>,
 <Item title:"JornadaStudy_sdebrisgantrysoploecho" type:Feature Layer Collection owner:gmaurer1>,
 <Item title:"Export_Output" type:Feature Layer Collection owner:gmaurer1>,
 <Item title:"GigaPan_Camera_Loc" type:Feature Layer Collection owner:gmaurer1>]

In [154]:
# Get metadata for a published item
my_content[0].download_metadata()

'/tmp/metadata.xml'

In [15]:
# Trying to import a pandas dataframe as an item
# This doesn't really work with geometry columns. If there were
# X & Y columns it might
test_fc = gis.content.import_data(test)
test_fc

/home/greg/data/miniconda3/envs/jgeo/lib/python3.8/site-packages/arcgis/features/geo/_io/fileops.py:1027: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  geom_type = df.loc[idx][geom_field].type


KeyError: 'MultiPolygon'

## Create 'jgeo_layers' and 'jgeo_layers_public' folders

In [136]:
# Create our content folders if they don't exist
gis.content.create_folder('jgeo_layers')
gis.content.create_folder('jgeo_layers_public')

{'username': 'gmaurer1',
 'id': 'df57116977dc44748f72bf7fc45aa7b2',
 'title': 'jgeo_layers_public'}

In [143]:
# Inventory of layers
jgeo_online = pd.read_csv(os.path.join(jgeo_path, 'jgeo_layers_online.csv'))
# Get layers that are in jgeo only
jgeo_online = jgeo_online[~pd.isna(jgeo_online.jgeo_table_name)]

In [150]:
import imp
imp.reload(agol)
imp.reload(pg)

for i in jgeo_online[60:].index:
    print(i)
    # Get jgeo layer info from table
    schema = jgeo_online.jgeo_schema[i]
    layer = jgeo_online.jgeo_table_name[i]
    print(layer)
    # Pull metadata from jgeo
    md = pg.md_dict(conn, schema, layer)
    fl_properties = agol.qmd_to_agol(jgeo_online.agol_title[i], md['qmd'])
    # Write to geojson file in temp dir
    geojson_fname = pg.table_to_geojson(conn, schema, layer, 
                                        outdir=os.path.join(jgeo_path, 'tmp'))
    # Publish the layer to jgeo_layers_public if requested
    if jgeo_online.make_public[i]:
        item, fl = agol.add_geojson_item(geojson_fname, gis,
                                         'jgeo_layers_public',
                                         properties=fl_properties,
                                         publish_after=True,
                                         remove_after=True,
                                        )
        # Delete the geojson file from agol
        item.delete()
        # Share with public
        fl.share(everyone=True, org=True,
            groups=[jgrp_id, jgrp_pub_id])
        
    # Or to jgeo_layers if for internal use
    else:
        item, fl = agol.add_geojson_item(geojson_fname, gis, 
                                         'jgeo_layers',
                                         properties=fl_properties,
                                         publish_after=True,
                                         remove_after=True)
        # Delete the geojson file from agol
        item.delete()
        fl.share(everyone=False, org=False,
            groups=[jgrp_id])
    
    #layer = 'prj404_cdrrcvegetation1938_new'
    #test = pg.table_to_gdf(conn, schema, layer)
    #md = pg.md_dict(conn, schema, layer)
#md = pd.read_sql("SELECT * FROM public.qgis_layer_metadata WHERE f_table_name = 'prj404_cdrrcvegetation1938_new';", conn)

72
prj405_jrnphysiography2000
73
prj405_jrnlandforms2000
74
prj405_jrnsoilparentmaterials2000
75
prj420_playas_30
76
prj420_watersheds_30
77
riogrande
78
isaacklakepoly


In [135]:
# Delete if we are just testing!
gis.content.delete_folder('jgeo_layers_public')
gis.content.delete_folder('jgeo_layers')

True

## Copy research project geopackages to AGOL

This is deprecated - just for example

In [50]:
# Metadata file
metadata = pd.read_csv(os.path.join(jgeo_path, 'metadata', 'JornadaGeosp_metadata_20220107.csv'))

# Get a sorted list of all the project geopackages
prj_gpkgs = sorted(glob.glob(os.path.join(jgeo_path, "jrn_studies", "/*.gpkg")))

# Create a folder for GeoJSON items and for the 
# packages (FeatureLayerCollections)
folder_geojson = 'JornadaResearchProjects_geojson'
gis.content.create_folder(folder_geojson)
folder_pkg = 'JornadaResearchProjects'
gis.content.create_folder(folder_pkg)

for i, pkg_path in enumerate(prj_gpkgs[0:6]):
    pkg_name = pkg_path.split('/')[1][0:6]
    studyid = pkg_name[3:6]
    print(pkg_name)
    # Get the metadata
    pkg_md = metadata[metadata.gpkg_fname==pkg_name+'.gpkg']
    
    # Create a hosted feature service on AGOL
    new_svc = gis.content.create_service(name=pkg_name,
        service_type='featureService', folder=folder_pkg)
    # Create a featureLayerCollection with new_svc
    new_flc = fs.FeatureLayerCollection.fromitem(new_svc)
    # Share the feature service with the JornadaGIS group
    new_svc.share(everyone=False, org=False, groups=[jornadagis_grpid])

    # Update the properties of the new featureLayerCollection (title,
    # summary, description, etc)
    fs_defs = agol.jornada_fs_definitions(new_flc, pkg_md)
    # A little kludge to fix the geopackage description
    fs_defs['description'] = fs_defs['description'].replace(
        ' layer is part of ', ' is ')
    new_flc.manager.update_definition(fs_defs)
    new_svc.update(fs_defs) # This updates the item info?
    # Copy the geopackage layers into the featureLayerCollection
    agol.geopkg_to_multi_fs(gis, folder_geojson, new_flc, pkg_path,
        pkg_md, clean_local=True)

    

prj001
True
True
True
True
True
prj002
True
prj004
True
prj008
True
prj009
True
True
prj010
True
True


In [49]:
folder_geojson = 'JornadaResearchProjects_geojson'
folder_pkg = 'JornadaResearchProjects'
gis.content.delete_folder(folder_geojson)
gis.content.delete_folder(folder_pkg)

Folder doesn't exist.
Folder doesn't exist.


False

## Copy thematic geopackages

This is deprecated - just for example

In [7]:
gpkgs = ['JornadaVegetation.gpkg',
    'JornadaBounds&Structures.gpkg',
    'JornadaMonitoringStations.gpkg',
    'JornadaLandforms&Soil.gpkg',
    'JornadaMisc.gpkg']

for i, pkg_path in enumerate(gpkgs[0:5]):
    # Create a folder
    folder_pkg = pkg_path.split('.')[0]
    gis.content.create_folder(folder_pkg)

    # Get the metadata
    pkg_md = metadata[metadata.package_name==folder_pkg]
    
    # Copy the geopackage layers into the featureLayerCollection
    items, fslist = agol.geopkg_to_single_fs(gis, folder_pkg, pkg_path,
        pkg_md, clean_local=True)

    # Share the feature layers with groups
    for i in range(len(fslist)):
        fslist[i].share(everyone=False, org=False,
            groups=[jornadagis_grpid, johntest_grpid])

CDRRCGrassCondition1858
CDRRCShrubPresence1858
JERGrassCondition1858
JERShrubPresence1858
JERVegetation1915
JERVegetation1928
JERVegetation1998
Buffington_Herbel_1858
Buffington_Herbel_1915
Buffington_Herbel_1928
Buffington_Herbel_1963
1938_CDRRC_Vegetation
1998_CDRRC_Vegetation
Boundary_JER_full
Boundary_CDRRC_JER
All3_Bdry_25Aug15
Airstrip
BurnPlots21Aug13
Cattleguards
CDNP_Bdry_25Aug15
CDRRC_Bdry_25Aug15
CDRRC_ExcloEDITED21Aug13
CDRRC_Exclosures_17Sep15
ClosedRoad
CSISParking
DefunctTowers
Dirt_Tanks
ExclosuresJER_17Jul2015
ExclusuresJER_Jul172015
JER_Bdry_25Aug15
JER_CDRRC_Bdry_FromCorrectedMar2014
JER_DeededBndry_OCt2013
JER_FtB_WSMR_bndry
Lvstk_Corral_Lot_Lane
Pastures_CDRRC2_16Sep15
Pastures_JER_16Sep15
RanchStructures
random_troughs
skipping layer: random_troughs
Rds_ClipToJER_27Apr18
RemvdFenceLabel
skipping layer: RemvdFenceLabel
RemvdFences
Tanks
Towers3
TwdieTower_WGS84
UTEP_Site_Access_Trail_UTM
Wells
AllRds_JER_And_Vicinity_04Sep15
Fence_JER_06Nov15
Fence_CDRRC_26Aug15
Jo

In [157]:
#folder_geojson = 'JornadaMonitoringStations'
#folder_pkg = 'JornadaResearchProjects'
#gis.content.delete_folder(folder_geojson)
#gis.content.delete_folder(folder_pkg)